This report investigates the supply and demand of New Zealand emission units, within New Zealand's Emission's trading scheme. 

It investigates historical demand, historical supply and analysis the holdings of the current stockpile.

If you enjoy it, or find it interesting, please let the [author](https://www.linkedin.com/in/alexbissell/) know (Linkedin messages are free).

The data used to generate this report is free and publicly available. 

You can access the data, and the notebook used to generate this report, from this [repository](https://github.com/bissella/NewZealandEmissions). 

In [1]:
import os

import pandas as pd
import plotly.express as pex
import plotly.graph_objects as go
from plotly.io import write_image
import numpy as np
from scipy.stats import linregress

data_folder = "./raw_data/"

# Historical Emissions

The figure below details New Zealand's historic gross and net emissions. 

Biogenic methane dominates the emission footprint making up ~49% of total emissions. 

This data was sourced from Stats New Zealand. 

The footprint from gross emitters is estimated by subtracting the methane emissions from the net emissions. 

This shows an approximate gross footprint of ~20 MtCO2e per year.

In [2]:
# plot historic emissions

raw_emissions = pd.read_csv("./raw_data/historical_emissions.csv")

# Fix the biogenic methane issue

biogenic_methane = raw_emissions[["year", "biogenic_ch4"]].drop_duplicates()
# Step 2: Create a new DataFrame with 'Biogenic CH4' as the 'key'
biogenic_ch4_rows = biogenic_methane.assign(key='Biogenic CH4', emissions=biogenic_methane['biogenic_ch4']).drop(columns='biogenic_ch4')
# Adding back to historic emisisons
historic_emissions = pd.concat([raw_emissions, biogenic_ch4_rows], ignore_index=True)
historic_emissions.drop(columns=['biogenic_ch4'], inplace = True)

historic_emissions_pivot = historic_emissions.pivot(index='year', columns = 'key', values = 'emissions').reset_index()
historic_emissions_pivot['ETS_Emission_Gap'] = historic_emissions_pivot['GHG emissions: net'] - historic_emissions_pivot['Biogenic CH4']

In [3]:
def label_every_n_points(data, n=5):
    # Create a new list of labels, with every nth label kept and the rest set to None
    return [label if index % n == 0 else None for index, label in enumerate(data)]

n = 5  # for example, label every 5th point
plot_columns = ['GHG emissions: gross',"GHG emissions: LULUCF"]
# Apply the function to your label list
gross_ghg_labels = label_every_n_points([f'{x:,.2f}M' for x in historic_emissions_pivot['GHG emissions: gross']], n)
gross_forestry_labels = label_every_n_points([f'{x:,.2f}M' for x in historic_emissions_pivot["GHG emissions: LULUCF"]], n)
net_ghg_labels = label_every_n_points([f'{x:,.2f}M' for x in historic_emissions_pivot['GHG emissions: net']], n)
biogenic_ch4_labels = label_every_n_points([f'{x:,.2f}M' for x in historic_emissions_pivot['Biogenic CH4']], n)
ets_emission_gap_labels = label_every_n_points([f'{x:,.2f}M' for x in historic_emissions_pivot['ETS_Emission_Gap']], n)

historic_emisisons_fig = pex.bar(data_frame=historic_emissions_pivot, x = 'year', y = plot_columns)

historic_emisisons_fig.add_trace(
  go.Scatter(x=historic_emissions_pivot['year'], y=historic_emissions_pivot['GHG emissions: gross'], name="Gross GHG", mode = 'text',
             text= gross_ghg_labels,
             showlegend=False,
             textposition='top center',
            )
)

historic_emisisons_fig.add_trace(
  go.Scatter(x=historic_emissions_pivot['year'], y=historic_emissions_pivot['GHG emissions: LULUCF'], name="Forestry", mode = 'text',
             text= gross_forestry_labels,
             showlegend=False,
             textposition='top center',
            )
)
  
historic_emisisons_fig.add_trace(
  go.Scatter(x=historic_emissions_pivot['year'], y=historic_emissions_pivot['GHG emissions: net'], name="Net GHG", mode = 'lines+text',
             text= net_ghg_labels,
             textposition='top center',
            )
)

historic_emisisons_fig.add_trace(
  go.Scatter(x=historic_emissions_pivot['year'], y=historic_emissions_pivot['Biogenic CH4'], name="Biogenic Methane", mode = 'lines+text',
             text= biogenic_ch4_labels,
             textposition='top center',
            )
)
historic_emisisons_fig.add_trace(
  go.Scatter(x=historic_emissions_pivot['year'], y=historic_emissions_pivot['ETS_Emission_Gap'], name="ETS Emissions", mode = 'lines+text',
             text= ets_emission_gap_labels,
             textposition='top center',
            )
)

historic_emisisons_fig.update_layout(title = "New Zealand Historical Emission Profile by source and year - Data from Stats NZ",
                                     yaxis_title = "tCO2e (million)"
)
historic_emisisons_fig.write_image("historic_emissions.png", width = 1920, height=1080, scale =2)
historic_emisisons_fig.show()

## Demand

In [4]:
demand_df = pd.read_csv(os.path.join(data_folder, 'participant_emission_report.csv')).rename(columns={"Reported emissions\n (tCO_{2} e)": "Reported Footprint tCO2e"})

# remove agiculture
demand_df = demand_df[demand_df['Sector']!= 'Agriculture']

# Replace commas and convert the '-' to NaN
demand_df["Reported Footprint tCO2e"] = demand_df["Reported Footprint tCO2e"].str.replace(',', '')
demand_df["Reported Footprint tCO2e"] = demand_df["Reported Footprint tCO2e"].replace('-', np.nan)

# Convert the column to numeric, coercing errors to NaN
demand_df["Reported Footprint tCO2e"] = pd.to_numeric(demand_df["Reported Footprint tCO2e"], errors='coerce')
demand_df = demand_df.dropna(subset=["Reported Footprint tCO2e"])

emisison_threshold = 500000
emission_filter = demand_df["Reported Footprint tCO2e"]>= emisison_threshold

demand_fig = pex.bar(data_frame=demand_df[emission_filter], x='Participant', y="Reported Footprint tCO2e", color='Sector')
# Change to log axis
demand_fig.update_layout(title = "New Zealands Top Non-Agriculatural Emitters (>500 kTCO2e) <br><sub>Data obtained from the EPA for 2023 reported emissions<sub>",
                         yaxis_type="linear")

demand_fig.show()
demand_fig.write_image("emitter_demand_top_emitters.png", width = 1920, height=1080, scale =2)

Top 30 emitters, neglecting agriculture.

In [5]:
demand_df.columns
demand_df.sort_values("Reported Footprint tCO2e", ascending=False).drop(columns=['Reported removals\n (tCO_{2} e)']).head(n=30)

,Participant,Sector,Activity,Schedule,Year,Reported Footprint tCO2e
255,Z Energy Limited,Liquid Fossil Fuels,Owning obligation fuel,Three,2022,7087002.0
145,Mobil Oil New Zealand Limited,Liquid Fossil Fuels,Owning obligation fuel,Three,2022,4623842.0
27,BP Oil New Zealand Ltd,Liquid Fossil Fuels,Owning obligation fuel,Three,2022,3957962.0
229,Todd Energy Limited,Stationary Energy,Mining natural gas,Three,2022,1556615.0
230,Todd Petroleum Mining Company Ltd,Stationary Energy,Mining natural gas,Three,2022,1117521.0
105,Gull New Zealand Limited,Liquid Fossil Fuels,Owning obligation fuel,Three,2022,1040686.0
171,OMV NZ Production Limited,Stationary Energy,Mining natural gas,Three,2022,882960.0
155,New Zealand Steel Limited,Stationary Energy,Purchasing coal,Four,2022,833247.0
47,Contact Energy Limited,Stationary Energy,Purchasing natural gas,Four,2022,784817.0
80,Fonterra Limited,Stationary Energy,Purchasing coal,Four,2022,736593.0


Reported emissions by sector:

In [6]:
summary = demand_df.groupby("Sector")["Reported Footprint tCO2e"].sum().reset_index()
fig = pex.pie(summary, values="Reported Footprint tCO2e", names="Sector", title = "2023 Reported Emisisons by Sector <br><sub>Data obtained by from the EPA</sub>")
fig.show()

fig.write_image("emission_source_pie.png", width = 1920, height=1080, scale =2)


# Stockpile

There is a stockpile of New Zealand Emission Units. 

Data was obtained from the Environmental Protection Agency. 

The figures below break down the stockpile by NZU type and account holder type. 

- ***Other*** account holders are participants without NZU liabilities. They are speculators participating in the market. 
- ***Participants*** are forestry owners and businesses that must submit emission reports.

Overall there is a surpless of 160M NZUs. Of which, ~100 m are forestry units and carry a liability to the land owner in the event the trees are lumbered or destroyed and not replanted.

If the supply of NZUs were stopped, this accounts for 8 to 3 years of NZU demand if foresty units are sold or not.

In [7]:
stockpile_df = pd.read_csv(os.path.join(data_folder, 'stockpile_data.csv')).rename(columns={"Holdings of NZU_AUC":"AuctionUnits"})

def convert_cells_to_integers(df:pd.DataFrame, columns:list):
    # Iterate over all columns in the DataFrame
    for col in columns:
        # Check if the column is of object type (string)
        if df[col].dtype == 'object':
            # Replace commas and convert the column to integers
            df[col] = df[col].str.replace(',', '').astype(int)
    return df

col_list = [col for col in stockpile_df.columns if col != 'Date']

df = convert_cells_to_integers(df=stockpile_df, columns=col_list)

df["ForestryNZUs"] =df['Holdings of NZU_FA'] + df['Holdings of NZU_FE and NZU_PFSI']

foo = df[['Date', 'All other NZUs', 'ForestryNZUs', 'AuctionUnits']] 

fig = pex.bar(data_frame=foo, x='Date', y = ['All other NZUs', "ForestryNZUs", 'AuctionUnits'], text_auto=True)

# marker_trace = pex.scatter(data_frame=df, x='Date', y='Total holdings of NZUs', text_auto=True)

fig.add_trace(

  go.Scatter(x=df['Date'], y=df['Total holdings of NZUs'], name="Total Holdings", 
             mode = 'text',
            # text_auto=True,
             text= [f'{x:,.2f}M' for x in df['Total holdings of NZUs']/1e6],
             textposition='top center',
             showlegend=False
            )
)

# fig.update_traces(texttemplate='%{text::.2f}')
fig.update_layout(title = "30th June NZU balance by NZU Type",
                  yaxis_title = "NZUS")


fig.show()
fig.write_image("Stockpile_with_forestry.png", width = 1920, height=1080, scale =2)


In [8]:
#| echo: false
goo = df[['Date', 'All other NZUs', 'AuctionUnits']]
goo["Total NonForestry NZUs"] = goo["All other NZUs"] + goo['AuctionUnits'] 

fig = pex.bar(data_frame=goo, x='Date', y = ['All other NZUs', 'AuctionUnits'], text_auto=True)

fig.add_trace(

  go.Scatter(x=goo['Date'], y=goo["Total NonForestry NZUs"], name="Total NonForestry NZUs", 
             mode = 'text',
            # text_auto=True,
             text= [f'{x:,.2f}M' for x in goo["Total NonForestry NZUs"]/1e6],
             textposition='top center',
             showlegend = False
            )
)

# fig.update_traces(texttemplate='%{text::.2f}')
fig.update_layout(title = "30th June NZU balance by NZU Type - neglecting forestry",
                  yaxis_title = "NZUS")

fig.show()

fig.write_image("stockpile_forestry_free.png", width = 1920, height=1080, scale =2)

/var/folders/h6/5nzk4gjx2_798h5phtpn167w0000gn/T/ipykernel_37416/2562780589.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Who is Holding What?

The Department of Environment have a category of participants, they defined as "Recieved NZUS" who are not participants. 

Their website defines these holders as:

***"individuals and organisations that have received NZUs from the Crown, but who are not participants. This includes, for example, some owners of pre-1990 forests, participants in the Permanent Forest Sink Initiative (PFSI), and organisations receiving an Industrial Allocation"***

I think this is silly because participants that receive industrial allocation need to report their emissions and surrender units. 
Similarily if a permanent forest sink initiative is destroyed, the owners are liable to pay the NZU debt or replant.

For this reason, I lumped "Received NZUs" holders into the same bucket as Participants.

In [9]:
# ACCOUNT HOLDERS CODE
holdings_df = pd.read_csv(os.path.join(data_folder, "nzu_holders_by_participant_type.csv"))
holdings_df = convert_cells_to_integers(holdings_df, columns = [col for col in holdings_df.columns if col != "Date" and col != "Account holder category"])
holdings_df["Holdings of Forestry NZUs"] = holdings_df["Holdings of NZU_FA"]+holdings_df["Holdings of NZU_FE and NZU_PFSI"]
holdings_df["Date"] = pd.to_datetime(holdings_df["Date"])
holdings_df = holdings_df.sort_values(by="Date", ascending=False)

holdings_df["Account holder category"] = holdings_df["Account holder category"].replace('Received NZUs', 'Participants')
# Group the Received NZUs category as particpants.
grouped_df = holdings_df.groupby(["Account holder category", "Date"]).sum().reset_index()

nzus_data_melted = grouped_df.melt(id_vars = ["Account holder category", "Date"],
                                    var_name='NZU Type',
                                    value_name="Holdings")

holdings_column = ["Holdings of Forestry NZUs", "Holdings of NZU_AUC", "All other NZUs"]

filtered_data = nzus_data_melted[nzus_data_melted["NZU Type"].isin(holdings_column)]


In [10]:
# ACCOUNT HOLDERS
stacked_holdings_plot = pex.area(data_frame=filtered_data, x="Date", y="Holdings",
                                 color = "NZU Type",
                                 facet_col = 'Account holder category',
                                 hover_data= ["NZU Type"],
                                 title = "NZU Holdings by Type and Account Holder Category",
                                 labels = {"Holdings": "NZU Holdings"},
                                 template = "none")

stacked_holdings_plot.update_traces(opacity=0.7)  # Adjust opacity
stacked_holdings_plot.write_image("NZU_holdings_forestry_assembled.png", width = 1920, height=1080, scale =2)
stacked_holdings_plot.show()

# Supply

## Removals

Forestry dominates the removal activity. 

Mandatory reporting every 5 years, leading to the 5 year spikes of removal units awarded.

Without forestry, there is approximately 3M NZUs generated annually through other removal mechanisms.

In [11]:

# Removal Activity

removal_df = pd.read_csv(os.path.join(data_folder, "removal.csv"))
removal_df = removal_df.rename(columns={"The_Year":"Year", "Number_Of_Units":"NZUs"})
removal_df.fillna(value="Forestry", inplace=True)

removal_bar = pex.bar(data_frame=removal_df, x='Year', y="NZUs", color= "Activity")
removal_bar.update_layout(title_text = "NZUs awarded by removal activity by year",
                          annotations=[
                            dict(
            xref='paper',
            yref='paper',
            x=0.21,
            y=1.01,
            xanchor='center',
            yanchor='bottom',
            text='Data from Emissions Trading Register',
            showarrow=False,
            font=dict(
                size=12
            )
        )
                          ]
)

aggregated_data = removal_df.groupby('Year')['NZUs'].sum().reset_index()

#line graph
removal_bar.add_trace(
  go.Scatter(x=aggregated_data['Year'], y=aggregated_data['NZUs'], name="Total NZUs", mode = 'text',
             text= [f'{x:,.2f}M' for x in aggregated_data['NZUs'] / 1e6],
             textposition='top center',
             showlegend=False
            )
)
removal_bar.write_image("historic_removal.png", width = 1920, height=1080, scale =2)
removal_bar.show()

In [12]:

# Assuming removal_df is your DataFrame and it's already defined
non_forestry_removal = removal_df[removal_df["Activity"] != "Forestry"]

# Create the scatter plot
non_forestry_scatter = pex.bar(data_frame=non_forestry_removal, x="Year", y="NZUs", color="Activity", text_auto=True)

# # Group by 'Activity' and calculate trendline for each
# for activity, group in non_forestry_removal.groupby('Activity'):
#     x = group['Year']
#     y = group['NZUs']
    
#     # Get the line of best fit
#     slope, intercept, r_value, p_value, std_err = linregress(x, y)
    
#     # Add the trendline to the scatter plot
#     non_forestry_scatter.add_traces(pex.line(x=x, y=slope*x + intercept, color_discrete_sequence=['black']).data)
    
#     # Add R² value annotation
#     non_forestry_scatter.add_annotation(
#         x='2015',
#         # max(group['Year']),
#         y=max(slope*x + intercept),
#         text=f'{activity}: R² = {r_value**2:.2f}',
#         showarrow=False,
#         yshift=10
#     )


non_forestry_scatter.update_layout(title = "Removal activity without NZU liability by activity type and time")
# Show the plot
non_forestry_scatter.show()

non_forestry_scatter.write_image("historic_removal_sans_forestry_bar.png", width = 1920, height=1080, scale =2)


## Industrial Allocation




The calculation of industrial allocation under the Climate Change Response Act in New Zealand is calculated as:

***Final Allocation Entitlement for the Year***

 $$FA = LA \times \sum(PDCT \times AB)$$
   - Where:
     - $FA$ = Final allocation entitlement for the year.
     - $LA$ = Level of assistance for the activity for the year.
     - $PDCT$ = Amount of each prescribed product from the eligible industrial activity produced in the preceding year.
     - $AB$ = Prescribed allocative baseline for the product.
   - The level of assistance $LA$ decreases annually after 2020 due to a phase-out rate.

In [13]:
# Your data as a list of lists
data = [
    ["2017/2018_Q1", 0, "3,070"],
    ["2017/2018_Q2", 168, "2,021"],
    ["2017/2018_Q3", "3,332,611", 0],
    ["2017/2018_Q4", "3,585,800", "5,610"],
    ["2018/2019_Q1", 0, 0],
    ["2018/2019_Q2", "1,938", 84],
    ["2018/2019_Q3", "5,071,488", 950],
    ["2018/2019_Q4", "2,793,699", 0],
    ["2019/2020_Q1", 763, "14,708"],
    ["2019/2020_Q2", 23, 0],
    ["2019/2020_Q3", "4,671,338", 40],
    ["2019/2020_Q4", "3,749,816", 0],
    ["2020/2021_Q1", 0, 17],
    ["2020/2021_Q2", 0, 0],
    ["2020/2021_Q3", "1,932,581", 0],
    ["2020/2021_Q4", "5,138,846", 0],
    ["2021/2022_Q1", 0, "120,439"],
    ["2021/2022_Q2", 0, 0],
    ["2021/2022_Q3", "1,741,514", 0],
    ["2021/2022_Q4", "4,082,560", 0],
    ["2022/2023_Q1", 0, 0],
    ["2022/2023_Q2", "8,539", 0],
    ["2022/2023_Q3", "1,995,658", 0],
    ["2022/2023_Q4", "3,810,044", 111],
    ["2023/2024_Q1", "1,130", 0]
]

# Convert the list of lists into a DataFrame
df = pd.DataFrame(data, columns=["date", "industrial_allocation_awarded", "industrial_allocation_repayment"])

# Convert the number strings with commas into integers
df['industrial_allocation_awarded'] = pd.to_numeric(df['industrial_allocation_awarded'].str.replace(',', ''), errors='coerce').fillna(0).astype(int)
df['industrial_allocation_repayment'] = pd.to_numeric(df['industrial_allocation_repayment'].str.replace(',', ''), errors='coerce').fillna(0).astype(int)

df["industrial allocation"] = df["industrial_allocation_awarded"] - df["industrial_allocation_repayment"]
df['year'] = df['date'].str[0:4]
df_grouped = df.groupby(['year']).sum().reset_index()

In [14]:
industrial_allocation_barplot = pex.bar(df_grouped, x="year", y="industrial allocation", text_auto=True, title = "Industrial allocatations by reporting year")
industrial_allocation_barplot.update_layout(yaxis_title = "NZUs")
industrial_allocation_barplot.show()
industrial_allocation_barplot.write_image("industrial_allocation.png", width = 1920, height=1080, scale =2)

## Auctions


NZUS are distributed through quarterly auctions. The following plot shows the reduction in auction base supply. 

This neglects the cost containment reserve volume.

In [15]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Your data
years = ['2023', '2024', '2025', '2026', '2027', '2028']
supply = [15, 14.1, 12.6, 10.7, 9.1, 7]
percentage_change = [None, -6.00, -10.64, -15.08, -14.95, -23.08]  # None for the first year as there's no change

# Create subplots and mention that we want to share the x-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add bar graph for NZU Supply on primary y-axis
fig.add_trace(
    go.Bar(x=years, y=supply, name='NZU Supply (MT)', text=supply),
    secondary_y=False,
)

# Add line graph for Percentage Change on secondary y-axis
fig.add_trace(
    go.Scatter(x=years, y=percentage_change, name='Percentage Change', mode='lines+markers+text', text=percentage_change),
    secondary_y=True,
)

# Add titles and labels
fig.update_layout(
    title_text="NZU Auction Supply excluding cost containment reserve by year"
)

# Set x-axis title
fig.update_xaxes(title_text="Year")

# Set y-axes titles
fig.update_yaxes(title_text="NZU Supply (Million)", secondary_y=False)
fig.update_yaxes(title_text="Year on Year Change (%)", secondary_y=True)

# Show the figure
fig.show()
fig.write_image("auction_volumes.png", width = 1920, height=1080, scale =2)